In [1]:
import requests

import pandas as pd

# Data Acquisition Exercises

## 1

Using the code from the lesson as a guide and the REST API from https://python.zgulde.net/api/v1/items as we did in the lesson, create a dataframe named items that has all of the data for items.

In [9]:
url = 'https://api.data.codeup.com'
endpoint = '/documentation'
response = requests.get(url + endpoint)
print(response.json()['payload'])


The API accepts GET requests for all endpoints, where endpoints are prefixed
with

    /api/{version}

Where version is "v1"

Valid endpoints:

- /stores[/{store_id}]
- /items[/{item_id}]
- /sales[/{sale_id}]

All endpoints accept a `page` parameter that can be used to navigate through
the results.



In [12]:
endpoint = '/api/v1/items'
response = requests.get(url + endpoint)
data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [16]:
payload = data['payload']
payload.keys()

dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [19]:
payload['next_page']

'/api/v1/items?page=2'

In [49]:
def get_data(url, endpoint, name):
    data = pd.DataFrame()
    
    while True:
        print(f'Reading page {endpoint}', end = '\r')
        contents = requests.get(url + endpoint).json()
        page_contents = pd.DataFrame(contents['payload'][name])
        data = pd.concat([data, page_contents])
        
        if not (next_page := contents['payload']['next_page']):
            break
            
        endpoint = next_page
        
    data = data.reset_index().drop(columns = 'index')
    print('Loading complete. Returning data.')
        
    return data

In [50]:
items = get_data('https://api.data.codeup.com', '/api/v1/items', 'items')
items.shape

Loading complete. Returning data.


(50, 6)

## 2

Do the same thing, but for stores (https://python.zgulde.net/api/v1/stores)

In [38]:
url = 'https://api.data.codeup.com/api/v1/stores'
data = requests.get(url).json()
data.keys()

dict_keys(['payload', 'status'])

In [40]:
data['payload'].keys()

dict_keys(['max_page', 'next_page', 'page', 'previous_page', 'stores'])

In [41]:
data['payload']['max_page']

1

In [51]:
stores = get_data('https://api.data.codeup.com', '/api/v1/stores', 'stores')
stores.shape

Loading complete. Returning data.


(10, 5)

## 3

Extract the data for sales (https://python.zgulde.net/api/v1/sales). There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [47]:
# Let's try the function

sales = get_data('https://api.data.codeup.com', '/api/v1/sales', 'sales')

Reading page /api/v1/sales?page=183


TypeError: 'tuple' object is not callable

In [48]:
sales.shape

(913000, 5)

## 4

Save the data in your files to local csv files so that it will be faster to access in the future.

In [52]:
items.to_csv('items.csv', index = False)
stores.to_csv('stores.csv', index = False)
sales.to_csv('sales.csv', index = False)

## 5

Combine the data from your three separate dataframes into one large dataframe.

In [53]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   item_brand  50 non-null     object 
 1   item_id     50 non-null     int64  
 2   item_name   50 non-null     object 
 3   item_price  50 non-null     float64
 4   item_upc12  50 non-null     object 
 5   item_upc14  50 non-null     object 
dtypes: float64(1), int64(1), object(4)
memory usage: 2.5+ KB


In [61]:
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   store_address  10 non-null     object
 1   store_city     10 non-null     object
 2   store_id       10 non-null     int64 
 3   store_state    10 non-null     object
 4   store_zipcode  10 non-null     object
dtypes: int64(1), object(4)
memory usage: 528.0+ bytes


In [55]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   item         913000 non-null  int64  
 1   sale_amount  913000 non-null  float64
 2   sale_date    913000 non-null  object 
 3   sale_id      913000 non-null  int64  
 4   store        913000 non-null  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 34.8+ MB


In [66]:
# item links to item_id and store links to store_id
df = sales.merge(items, how = 'inner', left_on = 'item', right_on = 'item_id', validate = 'm:1')
df = df.drop(columns = 'item_id')
df = df.merge(stores, how = 'inner', left_on = 'store', right_on = 'store_id', validate = 'm:1')
df = df.drop(columns = 'store_id')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 913000 entries, 0 to 912999
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   item           913000 non-null  int64  
 1   sale_amount    913000 non-null  float64
 2   sale_date      913000 non-null  object 
 3   sale_id        913000 non-null  int64  
 4   store          913000 non-null  int64  
 5   item_brand     913000 non-null  object 
 6   item_name      913000 non-null  object 
 7   item_price     913000 non-null  float64
 8   item_upc12     913000 non-null  object 
 9   item_upc14     913000 non-null  object 
 10  store_address  913000 non-null  object 
 11  store_city     913000 non-null  object 
 12  store_state    913000 non-null  object 
 13  store_zipcode  913000 non-null  object 
dtypes: float64(2), int64(3), object(9)
memory usage: 104.5+ MB


## 6

Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

In [67]:
df = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4383 entries, 0 to 4382
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         4383 non-null   object 
 1   Consumption  4383 non-null   float64
 2   Wind         2920 non-null   float64
 3   Solar        2188 non-null   float64
 4   Wind+Solar   2187 non-null   float64
dtypes: float64(4), object(1)
memory usage: 171.3+ KB


## 7

Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.

In [68]:
from acquire import *

In [77]:
get_open_power_systems_data().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4383 entries, 0 to 4382
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         4383 non-null   object 
 1   Consumption  4383 non-null   float64
 2   Wind         2920 non-null   float64
 3   Solar        2188 non-null   float64
 4   Wind+Solar   2187 non-null   float64
dtypes: float64(4), object(1)
memory usage: 171.3+ KB


In [78]:
load_data('items', use_cache = False).info()

Reading from API.
Loading complete. Returning data.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   item_brand  50 non-null     object 
 1   item_id     50 non-null     int64  
 2   item_name   50 non-null     object 
 3   item_price  50 non-null     float64
 4   item_upc12  50 non-null     object 
 5   item_upc14  50 non-null     object 
dtypes: float64(1), int64(1), object(4)
memory usage: 2.5+ KB
